In [ ]:
from pathlib import Path
import os, pandas as pd
from datasets import Dataset, load_from_disk
from transformers import AutoTokenizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

PROJECT_ROOT = Path.cwd()
DATA_IN = PROJECT_ROOT / "data" / "cleaned_data.csv"
DATA_OUT_DIR = PROJECT_ROOT / "data" / "tokenized_bert_uncased_max128"
MAX_LEN = 128
REMOVE_STOPWORDS = False

assert DATA_IN.exists(), f"Missing file: {DATA_IN}"


df = pd.read_csv(DATA_IN, usecols=["comment", "bias_sent", "category"]).dropna(subset=["comment","bias_sent"])

labels_num = pd.to_numeric(df["bias_sent"], errors="coerce")
mask = labels_num.isin([0, 1])
dropped = (~mask | labels_num.isna()).sum()
df = df.loc[mask, ["comment", "category"]].copy()
df["labels"] = labels_num.loc[mask].astype(int)


def maybe_remove_stopwords(text: str) -> str:
    if not REMOVE_STOPWORDS:
        return str(text)
    toks = [w for w in str(text).split() if w.lower() not in ENGLISH_STOP_WORDS]
    return " ".join(toks)

df["text"] = df["comment"].astype(str).apply(maybe_remove_stopwords)

print(f"Rows kept: {len(df)} | Dropped (invalid labels): {int(dropped)}")

# tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", use_fast=True)


ds = Dataset.from_pandas(df[["text", "labels", "category"]], preserve_index=False)

def tok(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=MAX_LEN)

ds = ds.map(tok, batched=True, desc="Tokenizing")

# keep standard columns
keep = ["input_ids", "attention_mask", "labels", "category"]
if "token_type_ids" in ds.column_names:
    keep.insert(1, "token_type_ids")
ds = ds.select_columns(keep)


DATA_OUT_DIR.mkdir(parents=True, exist_ok=True)
ds.save_to_disk(str(DATA_OUT_DIR))
print(f"Saved tokenized dataset to: {DATA_OUT_DIR}")


Rows kept: 11492 | Dropped (invalid labels): 47


c:\Code\MetaLanguageBias\language-bias\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\barif\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Saving the dataset (1/1 shards): 100%|██████████| 11492/11492 [00:00<00:00, 964501.08 examples/s] 


Saved tokenized dataset to: c:\Code\MetaLanguageBias\language-bias\data\tokenized_bert_uncased_max128
Example tokens: ['women', ',', 'doctors', ',', 'and', 'nurses', '!', '?']
Loaded OK. Columns: ['input_ids', 'token_type_ids', 'attention_mask', 'labels', 'category']
First row (truncated): {'input_ids': [101, 10047, 3374, 2023, 2003, 6230, 2005, 2017, 1012, 2012], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': 0, 'category': 'gender_female'}


In [5]:
loaded = load_from_disk(str(DATA_OUT_DIR))
print(loaded)  # or loaded["train"] if you saved a DatasetDict
row0 = {k: loaded[k][0] for k in ["labels","category"]}
print("Row 0 label/category:", row0)
max_len = max(len(ids) for ids in loaded["input_ids"]) if "input_ids" in loaded.column_names else 0
print("Max seq len:", max_len)  # should be <= 128


Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels', 'category'],
    num_rows: 11492
})
Row 0 label/category: {'labels': 0, 'category': 'gender_female'}
Max seq len: 128
